# Canonical occupations numbers and correlations for N bosons on a ring of L sites.

In [1]:
import numpy as np
import numpy.linalg as m
import pickle

In [2]:
def SpectrumPO(Δ,L,h,S,deg,BC):
    Nl=int((L)/2)
    if BC=='pbc':
        Size=Nl+1
        f=2
    if BC=='obc':
        Size=L
        f=1
    St=2*S+1
    ϵ=np.zeros((St,Size))
    K=np.zeros(Size)
    degeneracy=np.ones((St,Size))
    for k in range(Size):
        K[k]=f*(k+2-f)/L*np.pi
        for s in range(St):
            ϵ[s,k]=2*Δ*(1-np.cos(f*(k+2-f)/L*np.pi))-(s-2*S)*h
            degeneracy[s,k]=deg
    for s in range(St):
        degeneracy[s,0]=1
        if L/2==int(L/2) : 
            degeneracy[s,Nl]=1
    return ϵ,K,degeneracy

In [3]:
def AXSpectrumPO(Δ,L,h,S,deg,BC,i,si,j,sj,kk,skk,three):
    Nl=int((L)/2)
    if BC=='pbc':
        Size=Nl+1
        f=2
    if BC=='obc':
        Size=L
        f=1
    St=2*S+1
    ϵ=np.zeros((St,Size))
    K=np.zeros(Size)
    degeneracy=np.ones((St,Size))
    for k in range(Size):
        K[k]=f*(k+2-f)/L*np.pi
        for s in range(St):
            ϵ[s,k]=2*Δ*(1-np.cos(f*(k+2-f)/L*np.pi))-(s-S-S)*h
            degeneracy[s,k]=deg
    for s in range(St):
        degeneracy[s,0]=1
        if L/2==int(L/2) : 
            degeneracy[s,Nl]=1
    degeneracy[S+si,abs(i)]-=1
    degeneracy[S+sj,abs(j)]-=1
    if (three =='three'):
        degeneracy[S+skk,np.abs(kk)]-=1
    return ϵ,K,degeneracy


In [4]:
def Afactor(β,N,L,ϵ,degeneracy,S):
    St=2*S+1
    A=np.zeros(N+1)
    Nl=int((L)/2)
    for i in range(1,N+1):
        for k in range(Nl+1):
            for s in range(St):
                A[i]+=degeneracy[s,k]*np.exp(-1*β*ϵ[s,k]*i)
    return A

In [5]:
def PartitionFunctions(A,N,rescaleF=1e+300):
    rescaleC=0
    Z=np.zeros(N+1)
    Z[0]=1.0
    for i in range(1,N+1):
        for j in range(1,i+1):
            Z[i]+=A[j]*Z[i-j]/i
        if Z[i] > rescaleF:
            rescaleC+=1
            for k in range(i+1):
                Z[k]=Z[k]/rescaleF
    return Z,rescaleC                

In [6]:
def OccupNum(β,Z,ϵ,N,L,S):
    St=2*S+1
    Nl=int((L)/2)
    n=np.zeros((St,L))
    for i in range(1,N+1):
        for j in range(Nl+1):
            for s in range(St):
                n[s,Nl+j]+=np.exp(-1*β*ϵ[s,j]*i)*Z[N-i]/Z[N]
    for j in range(Nl+1):
        for s in range(St):
            n[s,L-1-(Nl+j)]=n[s,Nl+j]
    return n               

In [7]:
def ProbPB(L,N,Δ,h,S,deg,β,BC,i1,si1,j1,sj1,k1,sk1,three):
    ϵ_AX,K_AX,degeneracy_AX=AXSpectrumPO(Δ,L,h,S,deg,BC,i1,si1,j1,sj1,k1,sk1,three)
    AX_A=Afactor(β,N,L,ϵ_AX,degeneracy_AX,S)
    AX_Z,counter=PartitionFunctions(AX_A,N)
    #print('counterAX=',counter)
    ϵ,K,degeneracy=SpectrumPO(Δ,L,h,S,deg,BC)
    A=Afactor(β,N,L,ϵ,degeneracy,S)
    Z,counter=PartitionFunctions(A,N)
    #print('counter=',counter)
    prob=np.zeros((N+1,N+1))
    for i in range(0,N+1):
        for j in range(0,N+1):
            Nt=i+j
            if (Nt<N+1):
                prob[i,j]=np.exp(-1*β*(ϵ[si1+S,abs(i1)]*i+ϵ[sj1+S,abs(j1)]*j))*AX_Z[N-Nt]/Z[N]
            else:
                prob[i,j]=-1.0
    return prob

In [8]:
def nSquared(β,n,Z,ϵ,N,L,S):
    St=2*S+1
    Nl=int((L)/2)
    n2=np.zeros((St,L))
    for s in range(St):
        for j in range(Nl+1):
            for i in range(2,N+1):
                n2[s,Nl+j]+=(i-1)*np.exp(-1*β*ϵ[s,j]*i)*Z[N-i]/Z[N]
            n2[s,Nl+j]=2*n2[s,Nl+j]+n[s,Nl+j]
    for s in range(St):
        for j in range(Nl+1):
            n2[s,L-1-(Nl+j)]=n2[s,Nl+j]
    return n2               

In [9]:
# ninj=<n_in_j>, corr_ninj=<n_in_j>-<n_i><n_j> for i<>j and ϵ_i<>ϵ_j
def Correlations(s1,s2,S,β,ϵ,Z,n,N,L,h):
    St=2*S+1
    s1=s1+S
    s2=s2+S
    Nl=int((L)/2)
    σ2j=np.zeros(L)
    corr_ninj=np.zeros((L,L))
    ninj=np.zeros((L,L))    
    ni_nj=np.zeros((L,L))  
#First-quarter
    for i in range(0,Nl):
#--------upper triangle
        for j in range(i+1,Nl+1):
            corr_ninj[Nl+i,Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,i]))*n[s1,Nl+i]*(1+n[s2,Nl+j])-n[s2,Nl+j]*(1+n[s1,Nl+i]))/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,i]))-1)
            ninj[Nl+i,Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,i]))*n[s1,Nl+i]-n[s2,Nl+j])/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,i]))-1)
#--------lower triangle
        for j in range(i+1,Nl+1):
            if (abs(h)<1e-16):
                corr_ninj[Nl+j,Nl+i]=corr_ninj[Nl+i,Nl+j]
                ninj[Nl+j,Nl+i]=ninj[Nl+i,Nl+j]
            else:
                corr_ninj[Nl+j,Nl+i]=-1*(np.exp(-1*β*(ϵ[s2,i]-ϵ[s1,j]))*n[s1,Nl+j]*(1+n[s2,Nl+i])-n[s2,Nl+i]*(1+n[s1,Nl+j]))/(np.exp(-1*β*(ϵ[s2,i]-ϵ[s1,j]))-1)
                ninj[Nl+j,Nl+i]=-1*(np.exp(-1*β*(ϵ[s2,i]-ϵ[s1,j]))*n[s1,Nl+j]-n[s2,Nl+i])/(np.exp(-1*β*(ϵ[s2,i]-ϵ[s1,j]))-1)
#    if (s1 != s2):
    if (abs(h)<1e-16)|(s1 == s2):
        for j in range(0,Nl+1):
            for i in range(2,N+1):
                ninj[Nl+j,Nl+j]+=(i-1)*np.exp(-1*β*ϵ[s2,j]*i)*Z[N-i]/Z[N]
            corr_ninj[Nl+j,Nl+j]=ninj[Nl+j,Nl+j]-n[s1,Nl+j]*n[s2,Nl+j]
    else:
        for j in range(0,Nl+1):
            ninj[Nl+j,Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))*n[s1,Nl+j]-n[s2,Nl+j])/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))-1)
            corr_ninj[Nl+j,Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))*n[s1,Nl+j]*(1+n[s2,Nl+j])-n[s2,Nl+j]*(1+n[s1,Nl+j]))/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))-1)
#-copying First-quarter to all other quarters            
    for i in range(0,Nl+1):
        for j in range(0,Nl+1):
            corr_ninj[L-1-(Nl+i),L-1-(Nl+j)]=corr_ninj[Nl+i,Nl+j]
            corr_ninj[(Nl+i),L-1-(Nl+j)]=corr_ninj[Nl+i,Nl+j]
            corr_ninj[L-1-(Nl+i),(Nl+j)]=corr_ninj[Nl+i,Nl+j]
            ninj[L-1-(Nl+i),L-1-(Nl+j)]=ninj[Nl+i,Nl+j]
            ninj[(Nl+i),L-1-(Nl+j)]=ninj[Nl+i,Nl+j]
            ninj[L-1-(Nl+i),(Nl+j)]=ninj[Nl+i,Nl+j]
            
#fixing trailing diagonal in the first-quarter
    for j in range(1,Nl+1):
        ninj[L-1-(Nl+j),Nl+j]=0
        corr_ninj[L-1-(Nl+j),Nl+j]=0
    if (abs(h)<1e-16)|(s1 == s2):
        for j in range(1,Nl+1):
            for i in range(2,N+1):
                ninj[L-1-(Nl+j),Nl+j]+=(i-1)*np.exp(-1*β*ϵ[s2,j]*i)*Z[N-i]/Z[N]
            corr_ninj[L-1-(Nl+j),Nl+j]=ninj[L-1-(Nl+j),Nl+j]-n[s1,L-1-(Nl+j)]*n[s2,Nl+j]
    else:
        for j in range(1,Nl+1):
            ninj[L-1-(Nl+j),Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))*n[s1,L-1-(Nl+j)]-n[s2,Nl+j])/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))-1)
            corr_ninj[L-1-(Nl+j),Nl+j]=-1*(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))*n[s1,L-1-(Nl+j)]*(1+n[s2,Nl+j])-n[s2,Nl+j]*(1+n[s1,L-1-(Nl+j)]))/(np.exp(-1*β*(ϵ[s2,j]-ϵ[s1,j]))-1)
            
#copying trailing diagonal to theopposite quarter
     
    for j in range(0,Nl+1):
        ninj[Nl+j,L-1-(Nl+j)]=ninj[L-1-(Nl+j),Nl+j] 
        corr_ninj[Nl+j,L-1-(Nl+j)]=corr_ninj[L-1-(Nl+j),Nl+j] 
        
    for i in range(L):
        for j in range(L):
            ni_nj[i,j]=n[s1,i]*n[s2,j]
    return ninj,corr_ninj,ni_nj 

In [10]:
def CorrPB(s1,s2,L,N,Δ,h,S,deg,β,BC):
    ϵ,K,degeneracy=SpectrumPO(Δ,L,h,S,deg,BC)
    A=Afactor(β,N,L,ϵ,degeneracy,S)
    Z,counter=PartitionFunctions(A,N)
    n=OccupNum(β,Z,ϵ,N,L,S)
    ninj,corr_ninj,ni_nj=Correlations(s1,s2,S,β,ϵ,Z,n,N,L,h)
    #print('counter=',counter)
    return  corr_ninj,ninj,ni_nj,n              

In [11]:
def Corr4(ϵj,nj,β):
    A=np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            A[j,i]=np.exp(-i*ϵj[j]*β)
    Det1=m.det(A)
    for j in range(4):
        A[j,0]=nj[j]
    Det2=m.det(A)
    return  -Det2/Det1              

In [12]:
def Corr4d(ϵj,nj,β):
    A=np.zeros((4,4))
    for i in range(1,4):
        A[0,i]=np.exp(-i*ϵj[0]*β)
    for i in range(1,4):
        A[1,i]=np.exp(-i*ϵj[1]*β)
    for i in range(2,4):
        A[2,i]=(i-1)*np.exp(-i*ϵj[1]*β)
    for i in range(3,4):
        A[3,i]=(i-1)*(i-2)/2*np.exp(-i*ϵj[1]*β)

    for j in range(4):
        A[j,0]=1.0
    A[2,0]=-1.0
    Det1=m.det(A)
    for j in range(4):
        A[j,0]=nj[j]
    Det2=m.det(A)
    return  (-Det2/Det1) 


In [13]:
def Corr4d2(ϵj,nj,β,s1):
    A=np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            A[j,i]=np.exp(-i*ϵj[j]*β)

    A[s1+1,1]=0.0        
    for i in range(2,4):
        A[s1+1,i]=(i-1)*np.exp(-i*ϵj[0]*β)        
        
    for j in range(4):
        A[j,0]=1.0
    A[s1+1,0]=-1.0
    Det1=m.det(A)
    for j in range(4):
        A[j,0]=nj[j]
    Det2=m.det(A)
    return  (-Det2/Det1) 


In [14]:
# <n_{i,s1}n_{j,1}n_{j,0}n_{j,-1}>
def Correlations_4p(s1,s2,S,β,ϵ,Z,n,N,L,h):
    ϵj=np.zeros(4)
    nj=np.zeros(4)
    n1dotsn4=np.zeros((L,L))
    ni_nj=np.zeros((L,L))  
    St=2*S+1
    s1=s1+S
    s2=s2+S
    Nl=int((L)/2)
#First-quarter
    if (abs(h)<1e-16):
        for i in range(0,Nl):
#-----------upper triangle
            for j in range(i+1,Nl+1):
                ϵj[0]=ϵ[s1,i]
                ϵj[1]=ϵ[0,j]
                ϵj[2]=ϵ[1,j]
                ϵj[3]=ϵ[2,j]
                nj[0]=n[s1,Nl+i]
                nj[1]=n[0,Nl+j]
                nj[2]=0.0
                for m in range(2,N+1):
                    nj[2]+=(m-1)*np.exp(-1*β*ϵ[s2,j]*m)*Z[N-m]/Z[N]
                nj[3]=0.0
                for m in range(3,N+1):
                    nj[3]+=(m-1)*(m-2)/2.0*np.exp(-1*β*ϵ[s2,j]*m)*Z[N-m]/Z[N]
                n1dotsn4[Nl+i,Nl+j]=Corr4d(ϵj,nj,β)
        for i in range(1,Nl+1):
#-----------lower triangle
            for j in range(0,i):
                ϵj[0]=ϵ[s1,i]
                ϵj[1]=ϵ[0,j]
                ϵj[2]=ϵ[1,j]
                ϵj[3]=ϵ[2,j]
                nj[0]=n[s1,Nl+i]
                nj[1]=n[0,Nl+j]
                nj[2]=0.0
                for m in range(2,N+1):
                    nj[2]+=(m-1)*np.exp(-1*β*ϵ[0,j]*m)*Z[N-m]/Z[N]
                nj[3]=0.0
                for m in range(3,N+1):
                    nj[3]+=(m-1)*(m-2)/2.0*np.exp(-1*β*ϵ[0,j]*m)*Z[N-m]/Z[N]
                n1dotsn4[Nl+i,Nl+j]=Corr4d(ϵj,nj,β)
    else:              
        for i in range(0,Nl):
#-----------upper triangle
            for j in range(i+1,Nl+1):
                ϵj[0]=ϵ[s1,i]
                ϵj[1]=ϵ[0,j]
                ϵj[2]=ϵ[1,j]
                ϵj[3]=ϵ[2,j]
                nj[0]=n[s1,Nl+i]
                nj[1]=n[0,Nl+j]
                nj[2]=n[1,Nl+j]
                nj[3]=n[2,Nl+j]
                n1dotsn4[Nl+i,Nl+j]=Corr4(ϵj,nj,β)
        for i in range(1,Nl+1):
#-----------lower triangle
            for j in range(0,i):
                ϵj[0]=ϵ[s1,i]
                ϵj[1]=ϵ[0,j]
                ϵj[2]=ϵ[1,j]
                ϵj[3]=ϵ[2,j]
                nj[0]=n[s1,Nl+i]
                nj[1]=n[0,Nl+j]
                nj[2]=n[1,Nl+j]
                nj[3]=n[2,Nl+j]
                n1dotsn4[Nl+i,Nl+j]=Corr4(ϵj,nj,β)
            
    if (abs(h)<1e-16):
    
    
        for j in range(0,Nl+1):
            for i in range(4,N+1):
                n1dotsn4[Nl+j,Nl+j]+=(i-1)*(i-2)*(i-3)/6*np.exp(-1*β*ϵ[s2,j]*i)*Z[N-i]/Z[N]
            
    else:
        for j in range(0,Nl+1):
            ϵj[0]=ϵ[s1,j]
            ϵj[1]=ϵ[0,j]
            ϵj[2]=ϵ[1,j]
            ϵj[3]=ϵ[2,j]
            nj[0]=n[s1,Nl+j]
            nj[1]=n[0,Nl+j]
            nj[2]=n[1,Nl+j]
            nj[3]=n[2,Nl+j]
            nj[s1+1]=0.0
            for m in range(2,N+1):
                nj[s1+1]+=(m-1)*np.exp(-1*β*ϵ[s1,j]*m)*Z[N-m]/Z[N]
            n1dotsn4[Nl+j,Nl+j]=Corr4d2(ϵj,nj,β,s1)
#-copying First-quarter to all other quarters          
    for i in range(0,Nl+1):
        for j in range(0,Nl+1):
            n1dotsn4[L-1-(Nl+i),L-1-(Nl+j)]=n1dotsn4[Nl+i,Nl+j]
            n1dotsn4[(Nl+i),L-1-(Nl+j)]=n1dotsn4[Nl+i,Nl+j]
            n1dotsn4[L-1-(Nl+i),(Nl+j)]=n1dotsn4[Nl+i,Nl+j]
                  
    for i in range(L):
        for j in range(L):
            ni_nj[i,j]=n[s1,i]*n[0,j]*n[1,j]*n[2,j]
    return n1dotsn4,ni_nj 

# Parameters

In [15]:
β=np.array([1.0,100.0]) # inverse temperature
Δ=np.ones(len(β)) # energy levels separation

# Joint probability distributions

In [16]:
c=1
deg=2
L=1001
N=1000
Nl=int((L)/2)

S=0
h=0.0

i1=0
j1=1
k1=1

si1=0
sj1=0
sk1=0

prob2=ProbPB(L,N,Δ[c],h,S,deg,β[c],'pbc',i1,si1,j1,sj1,k1,sk1,'two')
prob3=ProbPB(L,N,Δ[c],h,S,deg,β[c],'pbc',i1,si1,j1,sj1,k1,sk1,'three')

In [17]:
structure = {'prob2F':prob2,'prob3F':prob3,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/probability.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

# Spin-zero two-point correlations

In [18]:
c=0
deg=2
L=1001
N=1000 
Nl=int((L)/2)

S=0
s1=0
s2=0
h=0.0

corr_ninj0p,ninj0p,ni_nj0p,n0p=CorrPB(s1,s2,L,N,Δ[c],h,S,deg,β[c],'pbc')

In [19]:
structure = {'corr_ninj0pF':corr_ninj0p,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin0L1001N1000tbeta1.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

# Spin-one two-point correlations

In [20]:
c=0
deg=2
L=1001
N=1000
Nl=int((L)/2)

S=1
s1=1
s2=1
h=0.0
corr_ninj1,ninj1,ni_nj1,n1=CorrPB(s1,s2,L,N,Δ[c],h,S,deg,β[c],'pbc')

S=1
s1=1
s2=1
h=5.0
corr_ninj1h11,ninj1h11,ni_nj1h11,n1h11=CorrPB(s1,s2,L,N,Δ[c],h,S,deg,β[c],'pbc')

S=1
s1=1
s2=0
h=5.0
corr_ninj1h10,ninj1h10,ni_nj1h10,n1h10=CorrPB(s1,s2,L,N,Δ[c],h,S,deg,β[c],'pbc')

S=1
s1=0
s2=-1
h=5.0
corr_ninj1h1neg1,ninj1h1neg1,ni_nj1h1neg1,n1h1neg1=CorrPB(s1,s2,L,N,Δ[c],h,S,deg,β[c],'pbc')

In [21]:
structure = {'corr_ninj1F':corr_ninj1,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin1L1001N1000tbeta1h0.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

In [22]:
structure = {'corr_ninj1h11F':corr_ninj1h11,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin1L1001N1000tbeta1h5s1s1.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

In [23]:
structure = {'corr_ninj1h10F':corr_ninj1h10,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin1L1001N1000tbeta1h5s1s0.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

In [24]:
structure = {'corr_ninj1h1neg1F':corr_ninj1h1neg1,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin1L1001N1000tbeta1h5s1s-1.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)

# Spin-one four-point correlations

In [25]:
c=0
deg=2
L=1001
N=1000
Nl=int((L)/2)

S=1
s1=1
s2=1
h=0.0

ϵ,K,degeneracy=SpectrumPO(Δ[c],L,h,S,deg,'pbc')
A=Afactor(β[c],N,L,ϵ,degeneracy,S)
Z,counter=PartitionFunctions(A,N)
n=OccupNum(β[c],Z,ϵ,N,L,S)
corr0,ni_nj0=Correlations_4p(s1,s2,S,β[c],ϵ,Z,n,N,L,h)

In [26]:
structure = {'corr0F':corr0,'NF':N,'LF':L,'betaF':β[c],'tF':Δ[c],'hF':h}

with open('../Data/CorrSpin1L1001N1000tbeta1_4pointst.pickle', 'wb') as pfile:
    pickle.dump(structure, pfile, pickle.HIGHEST_PROTOCOL)